The purpose of the code below is to download the latest COVID-19 data from 
Johns Hopkins University GitHub page, restructure it, clean it up a bit, 
and make some visualization using Plotly.

In [5]:
# All the imports in one place
import numpy as np
import pandas as pd
import os
import subprocess
import pycountry
import plotly.express as px

from pathlib import Path

To make sure that the charts are always made with the latest data the next
section launches a subprocess to run git and pull form the remote
repository.  It does the same thing as if you opened a terminal and ran:

`git pull https://github.com/CSSEGISandData/COVID-19.git`

This code assumes that your local COVID-19 repo is in the same parent folder of
as this repo, so prior to calling git the `os.chir` method is used to ensure
you are in the right folder.  (Don't worry, we come back home after everything
is loaded.)

The output message is printed to the screen to let you know if files were
updated or if you are already up to date.

In [6]:
# assumes git is installed
# assumes you are storing data in parent directory

os.chdir('../data/COVID-19')
cmd = ['git', 'pull', 'https://github.com/CSSEGISandData/COVID-19.git']
out = subprocess.run(cmd, stdout=subprocess.PIPE)
print(out.stdout.decode())

Already up to date.



We now have to define the path to the time series data and load them into
Pandas data frames.  JHU organizes the confirmed cases and deaths data
separately.  Here you see that the `Path` class was used from the `pathlib`
module.  This is a little extra, but it allows the code to be platform
agnostic.

Since we're done loading the JHU data we can go home again using the `os.chir`
command.  You may have to change this based on your structure and how you
name your project.

In [7]:
gl_confirmed_path = Path().joinpath('csse_covid_19_data',
                            'csse_covid_19_time_series',
                            'time_series_covid19_confirmed_global.csv')

gl_death_path = Path().joinpath('csse_covid_19_data',
                            'csse_covid_19_time_series',
                            'time_series_covid19_deaths_global.csv')

gl_c = pd.read_csv(gl_confirmed_path)
gl_d = pd.read_csv(gl_death_path)

os.chdir('../../scripts/')

For the charts I want to make I need to know the continent for each country. I
grabbed a file of countries and continents from
https://datahub.io/JohnSnowLabs/country-and-continent-codes-list and put it in
my data folder.

To build a dictionary of just the country names and its continent.  I do it by
1. Loading the `csv` into a data frame (`df_con`)
2. Create a dictionary
3. Iterate over `df_con` row by row using `itertuples`
4. Assign key/value pairs accessing the data with dot notation

There are some country names that are in the JHU data but not in the `csv`.  At
this point I decided to add the JHU names to this dictionary rather than change
the JHU names.  For a different purpose I'll change JHU the names to conform
with ISO standards, but for now I'll just update the dictionary I just loaded.

In [8]:
df_con = pd.read_csv('../data/country-and-continent-codes-list.csv')
continent_map = dict()
for row in df_con.itertuples():
    continent_map[row.Country_Name] = row.Continent_Name
continent_map.update({'US': 'North America', 'UK': 'Europe',
                      'Cabo Verde': 'Africa', 'Congo (Brazzaville)': 'Africa',
                      'Congo (Kinshasa)': 'Africa', 'Czechia': 'Europe',
                      'Diamond Princess': 'Asia', 'Eswatini': 'Africa',
                      'Korea, South': 'Asia', 'Kyrgyzstan': 'Asia',
                      'North Macedonia': 'Europe', 'Taiwan*': 'Asia',
                      'Laos': 'Asia', 'West Bank and Gaza': 'Asia',
                      'Kosovo': 'Europe', 'Burma': 'Asia',
                      'MS Zaandam': 'North America'})


This next section actually does a lot of heavy lifting.  This code is needed to
get the data in a structure that's way easier for making charts.  In JHU time
series files they add a column with numbers for that day.
It looks like this:

In [9]:
gl_c.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1828,1939,2171,2335,2469,2704,2894,3224,3392,3563
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,750,766,773,782,789,795,803,820,832,842
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,3649,3848,4006,4154,4295,4474,4648,4838,4997,5182
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,743,743,745,745,747,748,750,751,751,752
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,27,27,27,30,35,35,35,36,36,36


To get it in the structure I want it in I
 1. Group it by *Country/Region* `gl_c.groupby('Country/Region')`
 2. Add up the numbers to get the all the states/provinces for that country:
   `.sum()`
 3. Slice it to exclude the *Lat* and *Long* columns: `.iloc[:, 2:]`
 4. Unstack it (similar result as transpose): `.unstack()`
 5. Reset the index to put the country name on each row: `.reset_index()`

It end up looking like this:

In [10]:
c = gl_c.groupby('Country/Region').sum().iloc[:, 2:].unstack().reset_index()
c.tail(5)

,level_0,Country/Region,0
20004,5/7/20,West Bank and Gaza,375
20005,5/7/20,Western Sahara,6
20006,5/7/20,Yemen,25
20007,5/7/20,Zambia,153
20008,5/7/20,Zimbabwe,34


Next we do the same transform on the death data frame and combine the two
frames merging on the date and country.  Then take care of some housekeeping by
renaming the the columns to be more descriptive and useful.

In [11]:
d = gl_d.groupby('Country/Region').sum().iloc[:, 2:].unstack().reset_index()
df = pd.merge(c, d, on=['level_0', 'Country/Region'])
new_names = {'level_0': 'Date', 'Country/Region': 'Country',
             '0_x': 'Confirmed', '0_y': 'Deaths'}
df = df.rename(columns=new_names)

Reformat the *Date* to be standardized.

In [12]:
df['Date'] = pd.to_datetime(df['Date']).dt.strftime("%Y-%m-%d")

Combine all rows that have the same country name and date. (This might not
actually be needed, but I probably added it to fix something that was broken.)

In [13]:
df = df.groupby(['Date', 'Country']).sum().reset_index() # combine duplicate rows

Create the *Death Rate* percentage, replace NAN values with 1, and round to two
decimal places.

In [14]:
df['Death Rate'] = (df.Deaths / df.Confirmed * 100).fillna(1).round(2)

One of my favorite ways to set values in Pandas is to first get make an index
of all the rows that match the conditions for the change I need to make.  Then
I use the `.at` method to set the value I want.

Below I make a bubble chart and the size of the bubble is based on the *Death
Rate*.  If no one has died yet then the size of the bubble is zero, then you
can't see the bubble at all.  Here I find all the rows where *Death Rate* is
zero and change it to one.

In [15]:
idx = df[df['Death Rate'] == 0].index
df.at[idx, 'Death Rate'] = 1  # controls the bubble size

Here is a little function that is used to add the continent name to
the main data frame.  It takes the country name and returns which continent it
is on.  It has a little bit of lazy search by looking for the country in the
key.

This is a good example of how to use the `.apply` method in Pandas.  It was
after using this function that I learned which *key/value* pairs I have to add
to the `continent_map` dictionary.

In [16]:
def find_continent(country):
    for key, value in continent_map.items():
        if country in key:
            return value

df['Continent'] = df.Country.apply(find_continent)  # assign continent

I found that in the data there are sometime negative entries.  Think these were
reported to correct previously over-reported numbers, but I'm not really sure.
I clean this up quick and dirty by finding the negative values and setting them
to zero.  If in fact negative numbers are reported to correct the data, then
my numbers will be higher than other. It might be worth going back and fixing
the number correctly by modifying the previous value. But for now, this will
do.

In [17]:
idx = df[df['Confirmed'] < 0].index
df.at[idx, 'Confirmed'] = 0 # no negative cases allowed

idx = df[df['Deaths'] < 0].index
df.at[idx, 'Deaths'] = 0 # no negative deaths allowed

# people on Antarctica are social distancing enough.
idx = df[df.Continent == 'Antarctica'].index
df.drop(idx, inplace=True)


Using the [choropleth](https://plotly.com/python/choropleth-maps/) library in
Plotly requires that the countries are described by their 3-letter
abbreviation.  I have identified the main offenders and build a
`country_correction` dictionary with the country names that I know will be
recognized by the `pycountry` package that I use to lookup the ISO `alpha_3`.

I don't like nest `try except` statements but I couldn't figure out a cleaner
way to take care of this.  Let me know if you have a better way.
(richard.payton@lytworx.com)

In [18]:
country_correction = {
    'Burma': 'Myanmar',
    'Congo (Brazzaville)': 'Republic of the Congo',
    'Congo (Kinshasa)': 'Congo, The Democratic Republic of the',
    'Korea, South': 'Korea, Republic of',
    'Laos': "Lao People's Democratic Republic",
    'Taiwan*': 'Taiwan',
    'West Bank and Gaza': 'Palestine, State of',
    }
for key, value in country_correction.items():
    idx = df[df.Country == key].index
    df.at[idx, 'Country'] = value

def assign_alpha(x):
    try:
        a = pycountry.countries.get(name=x).alpha_3
        return a
    except AttributeError:
        try:
            a = pycountry.countries.get(common_name=x).alpha_3
            return a
        except AttributeError:
            try:
                a = pycountry.countries.search_fuzzy(x)
                a = a[0].alpha_3
                return a
            except LookupError:
                print(f'No country data for {x}.')
                return x

for c in df.Country.unique():
    idx = df[df.Country == c].index
    df.at[idx, 'iso_alpha_3'] = assign_alpha(c)

No country data for Diamond Princess.
No country data for MS Zaandam.


In order to set my scale on some of the color coded plots I need to know the
max values for each country over the entire time span.  I loop through all the
unique country names, find the max value and append to the `max_list`.  This is
useful for setting the scale to max at the 99th percentile so that the
numbers in the United States don't blow out the scale.

In [19]:
max_list = list()
for c in df.Country.unique():
    max_list.append(df.Confirmed[df.Country == c].max())

Now we are ready to make plots!  This will create a choropleth (shaded map)
that shows the number of confirmed cases.  It is an animated figure that
progresses from 14 Feb 2020 to whenever the data ends. Notice that the Plotly
title knows how to parse HTML (and my use of `f` strings.)

In [20]:
q = 99
cmax = int(np.percentile(max_list, q))

fig = px.choropleth(
    df[df.Date >= '2020-02-14'],
    locations="iso_alpha_3",
    color="Confirmed",
    animation_frame='Date',
    animation_group='Country',
    hover_name="Country",
    color_continuous_scale=px.colors.diverging.Portland,
    range_color=[0,cmax],
    projection='natural earth',
    title=f'COVID-19 Confirmed Cases (scale maxed at {q}th percentile: {cmax:,})<br>'
          f'Source:<a href="https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data">'
          f'JHU CSSE COVID-19 Dataset</a>'
    )
# fig.show()
fig.write_html('../charts/global_confirmed_cases_map.html')

This will create an animated bubble chart that shows the number of deaths
versus the number of cases.  The size of the bubble is determined by the death
rate percentage calculated above.

In [21]:
days = df.Date.unique()
fig = px.scatter(
    data_frame=df,
    x='Confirmed',
    y='Deaths',
    animation_frame='Date',
    animation_group='Country',
    size='Death Rate',
    color='Continent',
    hover_name='Country',
    size_max=100,
    title=f'COVID-19 Confirmed Cases<br>'
          f'Source:<a href="https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data">'
          f'JHU CSSE COVID-19 Dataset</a>',
    category_orders={'Day':days}
)
fig.update_layout(width=1200)
# fig.show()
fig.write_html("../charts/global_confirmed_cases_bubble_chart.html")

This is another figure with bubble charts, using all the same data as the
previous figure, but making a separate chart for each continent.

In [22]:
fig = px.scatter(
    data_frame=df,
    x='Confirmed',
    y='Deaths',
    animation_frame='Date',
    animation_group='Country',
    size='Death Rate',
    color='Country',
    hover_name='Country',
    facet_col='Continent',
    facet_col_wrap=3,
    size_max=75,
    title=f'COVID-19 Confirmed Cases<br>'
          f'Source:<a href="https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data">'
          f'JHU CSSE COVID-19 Dataset</a>',
    category_orders={'Day':days}
)
fig.update_layout(width=1200)
# fig.show()
fig.write_html("../charts/global_confirmed_cases_bubble_chart_per_continent.html")

In [24]:
def split_by_state(country, df):
    df = df[df.Country == country].copy()
    return df

def calc_growth_rate(df):
    #calc growth rate
    df['today'] = df.Confirmed.diff().fillna(0)
    df['yesterday'] = df.today.shift(1).fillna(method='ffill')
    df['growth_rate'] = (df['today'] / df['yesterday'] - 1).round(3)
    df['growth_rate'] = df['growth_rate'].replace([np.inf, -np.inf], np.nan)
    df['growth_rate'] = df['growth_rate'].fillna(method='ffill')
    # calc rolling growth rate
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date', drop=False)
    df = df.sort_index()
    df['rolling_growth_rate'] = df['growth_rate'].clip(-5,5).rolling('14d').mean().round(3)
    df['Date'] = df['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    return df

df_c = list()
for c in df.Country.unique():
    df_c.append(split_by_state(c, df))

new_dfs = list()
for d in df_c:
    new_dfs.append(calc_growth_rate(d))

sdn = pd.concat(new_dfs, ignore_index=True)
sdn.reset_index(drop=True)
sdn.to_csv('../data/global_confirmed_growth_rate.csv', index=False)

In [25]:
fig = px.choropleth(
    sdn[(sdn.Date >= '2020-01-01') &
        (pd.to_datetime(sdn.Date) <= pd.to_datetime(sdn.Date.max()) - pd.Timedelta(days=1))],
    locations='iso_alpha_3',
    color='rolling_growth_rate',
    animation_frame='Date',
    animation_group='Country',
    hover_name="Country",
    hover_data=['Date', 'Confirmed', 'Deaths', 'today', 'yesterday',
                'growth_rate'],
    color_continuous_scale=px.colors.diverging.RdYlGn_r,
    color_continuous_midpoint=0,
    range_color=[-1,1],
    projection='natural earth',
    title=f'COVID-19 Confirmed Cases Rolling 14-Day Average Growth Rate<br>'
          f'Source: <a href="https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data">'
          f'JHU CSSE COVID-19 Dataset</a>'
    )
fig.update_layout(
    coloraxis_colorbar=dict(
        title='Rolling Growth Rate'
        )
    )
fig.write_html('../charts/global_confirmed_cases_rolling_14-Day_average_growth_rate_map.html')

In [26]:
sdn[['Date', 'Confirmed', 'today', 'yesterday']][sdn.Country == 'US'].tail(20)

,Date,Confirmed,today,yesterday
18491,2020-04-18,732197,32491.0,32114.0
18492,2020-04-19,758809,26612.0,32491.0
18493,2020-04-20,784326,25517.0,26612.0
18494,2020-04-21,811865,27539.0,25517.0
18495,2020-04-22,840351,28486.0,27539.0
18496,2020-04-23,869170,28819.0,28486.0
18497,2020-04-24,905358,36188.0,28819.0
18498,2020-04-25,938154,32796.0,36188.0
18499,2020-04-26,965785,27631.0,32796.0
18500,2020-04-27,988197,22412.0,27631.0


In [27]:
# little data backup never hurt anyone...
df.to_csv('../data/scatter_global.csv', index=False)